In [92]:
##Python Dependencies
import pandas as pd
import json
import os
from flask import Flask, redirect, url_for, request

##Read Model Results
os.chdir("./result_data/")
cos_sim_name = pd.read_csv("Cosine_Sim_Name.csv",index_col=0)
cos_sim_score = pd.read_csv("Cosine_Sim_Score.csv",index_col=0)
corr_name = pd.read_csv("Corr_Name.csv",index_col=0)
corr_score = pd.read_csv("Corr_Score.csv",index_col=0)
os.chdir("../")

##Read Product Data
os.chdir("./data/data/")
meta_json = open("meta.json",encoding="utf8")
meta = json.load(meta_json)
df_meta = pd.json_normalize(meta["meta"])
df_meta = df_meta.set_index("productid")
os.chdir("../../")

##Define Flask Variable
app = Flask(__name__)

##Define endpoint to return results 
@app.route('/api/<name>')
def api(name):
    cos_product = cos_sim_name.loc[str(name),:].values.tolist()
    cos_score = cos_sim_score.loc[str(name),:].values.tolist()
    df_cos = df_meta.loc[cos_product]
    df_cos["score"] = cos_score
    df_cos.index = df_cos.index.rename("Cosine Similarity")
    html_cos = df_cos.to_html(header="true", table_id="table")
    
    cor_product = corr_name.loc[str(name),:].values.tolist()
    cor_score = corr_score.loc[str(name),:].values.tolist()
    df_cor = df_meta.loc[cor_product]
    df_cor["score"] = cor_score
    df_cor.index = df_cor.index.rename("Correlation")
    html_cor = df_cor.to_html(header="true", table_id="table")

    return '{} "\n\n" {}'.format(html_cos, html_cor)

##Add GET and POST request in the decorator
@app.route('/login',methods = [ 'GET', 'POST'])
def login():
   if request.method == 'GET':
      user = request.args.get('nm')
      return redirect(url_for('api',name = user))
   else:
      user = request.form['nm']
      return redirect(url_for('api',name = user))
    
##Run the flask variable
if __name__ == '__main__':
   app.run(debug = False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Aug/2021 23:00:09] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [08/Aug/2021 23:00:09] "GET /api/ZYHPCOCACCOL016 HTTP/1.1" 200 -
